In [4]:
import mne
import numpy as np
import os.path as op
from tools import files
from scipy.stats import trim_mean, ttest_1samp, sem
%matplotlib qt

In [5]:
main_path = op.join("/cubric", "scratch", "c1557187", "act_mis")
print(op.exists(main_path))
meg_path = op.join(main_path, "MEG")
beh_path = op.join(main_path, "BEH")
res_means = op.join(main_path, "RESULTS", "means_erf_occ.npy")

True


In [7]:
participants = files.get_folders_files(meg_path, wp=False)[0]
participants.sort()
participants.remove('0042')
print(participants)

['0001', '0003', '0004', '0008', '0009', '0014', '0015', '0016', '0017', '0019', '0021', '0027', '0030', '0032', '0034', '0035', '0038', '0039', '0040', '0041', '0043']


In [8]:
# 110 - "30"; 30 - "40"
cond = ["regular", "odd", "odd-regular", "regular-odd"] 

data = {i: [] for i in cond}

In [ ]:
# single instance of the loop

for pp in participants:
    participant_path = op.join(meg_path, pp)
    participant_files = files.get_files(participant_path, "all", "-epo.fif")[2]

    for i in participant_files:
        epochs = mne.read_epochs(i, preload=True)
        epochs = epochs.crop(tmin=-0.5, tmax=2.5)
        print(epochs)
        occipital_sensors = [i for i in epochs.info["ch_names"] if "O" in i]
        epochs = epochs.pick_channels(occipital_sensors)
        epochs = epochs.filter(0, 40, n_jobs=-1)

        reg = list(epochs["30"].iter_evoked())
        reg_info = reg[0].info
        reg = np.array([i.data for i in reg])
        reg_tm = trim_mean(reg, 0.05, axis=0)
        reg = mne.EvokedArray(reg_tm, reg_info, tmin=-0.5, comment="regular")
        odd = list(epochs["40"].iter_evoked())
        odd_info = odd[0].info
        odd = np.array([i.data for i in odd])
        odd_tm = trim_mean(odd, 0.05, axis=0)
        odd = mne.EvokedArray(odd_tm, odd_info, tmin=-0.5, comment="odd")
        subt0 = mne.combine_evoked([-reg, odd], weights="equal")
        subt1 = mne.combine_evoked([reg, -odd], weights="equal")
        reg.crop(tmin=1.4)
        odd.crop(tmin=1.4)
        subt0.crop(tmin=1.4)
        subt1.crop(tmin=1.4)
        reg = trim_mean(reg.data, 0.05, axis=0)
        odd = trim_mean(odd.data, 0.05, axis=0)
        subt0 = trim_mean(subt0.data, 0.05, axis=0)
        subt1 = trim_mean(subt1.data, 0.05, axis=0)

        data["regular"].append(reg)
        data["odd"].append(odd)
        data["odd-regular"].append(subt0)
        data["regular-odd"].append(subt1)

<EpochsFIF  |   140 events (all good), -0.5 - 2.5 sec, baseline [-0.5, 0], ~220.4 MB, data loaded,
 '30': 110
 '40': 30>
<EpochsFIF  |   140 events (all good), -0.5 - 2.5 sec, baseline [-0.5, 0], ~220.4 MB, data loaded,
 '30': 110
 '40': 30>


In [63]:
data["regular"] = np.vstack(data["regular"])
data["odd"] = np.array(data["odd"])
data["odd-regular"] = np.array(data["odd-regular"])
data["regular-odd"] = np.array(data["regular-odd"])

In [66]:

np.save(res_means, data)

In [29]:
data = np.load(res_means)

In [30]:
data = data.item()

In [31]:
results = np.apply_along_axis(ttest_1samp, 0, data["odd-regular"], 0)

In [32]:
results.shape

(2, 276)

In [43]:
import matplotlib.pylab as plt

time = np.linspace(-0.1,1, num=276)

mean = trim_mean(data["odd-regular"], 0.05, axis=0)
sem_ = sem(data["odd-regular"])

plt.plot(time, mean)
plt.fill_between(time, mean-sem_, mean+sem_, alpha=0.4)
plt.axhline(0)
plt.axvline(0)